In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)
        self.fc4 = nn.Linear(3, 64)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_size)
        ctr = x[:, -1]
        ctr_feature = self.fc4(ctr)
        
        x = x.view(-1, 60)
        batch_size = x.size(0)
        sequence_length = x.size(1)
        x = x.view(batch_size, sequence_length, 1)
        
        # Apply LSTM
        h0 = torch.zeros(2, batch_size, 64).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, batch_size, 64).to(x.device)  # Initial cell state
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Reshape and pass through linear layers
        lstm_out = lstm_out[:, -1, :]  # Take the last time step output
        x = self.relu(lstm_out)
        x = self.fc1(x)
        result_tensor = torch.cat((x, ctr_feature), dim=1)

        x = self.relu(result_tensor)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        x = F.softmax(x, dim=1)
        
        return x

model = Net()

In [6]:
# 预训练
pretrained_weights_path = 'model_weights.pth'
pretrained_weights = torch.load(pretrained_weights_path)
model.load_state_dict(pretrained_weights)
model.eval().cuda()

Net(
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=64, bias=True)
  (relu): ReLU()
)

In [11]:
test_data = pickle.load(open('test_data.pkl', "rb"))
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

print(len(test_dataloader.dataset))

device = 'cuda'
loss_fn = nn.CrossEntropyLoss()
test_loss, correct = 0, 0


real_label, pred_label = [], []
with torch.no_grad():
    for trajs, labels in test_dataloader:
        trajs, labels = trajs.to(device).float(), labels.to(device).float()
        pred = model(trajs)

        test_loss += loss_fn(pred, labels)
        _, predicted_labels = torch.max(pred, dim=1)
        _, labels = torch.max(labels, dim=1)
        
        real_label.extend(labels.tolist())
        pred_label.extend(predicted_labels.tolist())

        correct += (predicted_labels == labels).sum().item()
print(correct/len(test_dataloader.dataset))

105
0.9523809523809523


In [12]:
print(real_label)
print(pred_label)

[0, 0, 0, 2, 2, 2, 1, 1, 0, 1, 1, 0, 2, 0, 0, 1, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 2, 2, 2, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 0, 2, 0, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
